In [ ]:
import yt_dlp
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, widgets, Layout
import browser_cookie3
import subprocess
import http.cookiejar

# Global variable to store cookies
YOUTUBE_COOKIES = None

def login_youtube():
    global YOUTUBE_COOKIES
    
    browser_options = ['chrome', 'firefox', 'edge', 'safari']
    browser_dropdown = widgets.Dropdown(options=browser_options, description='Browser:')
    login_button = widgets.Button(description='Get Cookies')
    output = widgets.Output()

    def on_button_clicked(b):
        global YOUTUBE_COOKIES
        with output:
            output.clear_output()
            print(f"Fetching cookies from {browser_dropdown.value}...")
            try:
                if browser_dropdown.value == 'chrome':
                    YOUTUBE_COOKIES = browser_cookie3.chrome(domain_name='.youtube.com')
                elif browser_dropdown.value == 'firefox':
                    YOUTUBE_COOKIES = browser_cookie3.firefox(domain_name='.youtube.com')
                elif browser_dropdown.value == 'edge':
                    YOUTUBE_COOKIES = browser_cookie3.edge(domain_name='.youtube.com')
                elif browser_dropdown.value == 'safari':
                    YOUTUBE_COOKIES = browser_cookie3.safari(domain_name='.youtube.com')
                
                # Save cookies in Netscape format
                with open('youtube_cookies.txt', 'w') as f:
                    f.write("# Netscape HTTP Cookie File\n")
                    for cookie in YOUTUBE_COOKIES:
                        f.write(f"{cookie.domain}\tTRUE\t{cookie.path}\t"
                                f"{'FALSE' if cookie.expires is None else 'TRUE'}\t"
                                f"{cookie.expires if cookie.expires is not None else 0}\t"
                                f"{cookie.name}\t{cookie.value}\n")
                
                print("Cookies fetched and saved successfully!")
                print("Login process completed. You can now use these cookies for downloads.")
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                YOUTUBE_COOKIES = None

    login_button.on_click(on_button_clicked)
    display(browser_dropdown, login_button, output)

def check_ffmpeg():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except FileNotFoundError:
        return False

def download_video(url, quality='best'):
    output = widgets.Output()
    
    with output:
        ffmpeg_available = check_ffmpeg()
        if not ffmpeg_available:
            print("FFmpeg is not installed. Some high-quality formats may not be available.")
        
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best' if ffmpeg_available else 'best[ext=mp4]',
            'outtmpl': '%(title)s.%(ext)s',
            'writesubtitles': True,
            'cookiefile': 'youtube_cookies.txt'
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                # Check if the URL is a channel videos page
                if '@' in url and '/videos' in url:
                    print(f"Detected channel videos URL: {url}")
                    
                    # First, extract video URLs
                    channel_info = ydl.extract_info(url, download=False)
                    if 'entries' in channel_info:
                        video_urls = [f"https://www.youtube.com/watch?v={entry['id']}" for entry in channel_info['entries']]
                        print(f"Found {len(video_urls)} videos in the channel.")
                        
                        # Now download each video
                        for i, video_url in enumerate(video_urls, 1):
                            print(f"\nDownloading video {i} of {len(video_urls)}")
                            ydl.download([video_url])
                    else:
                        print("No videos found in the channel.")
                else:
                    # Handle single video or playlist as before
                    info = ydl.extract_info(url, download=False)
                    print(f"Video title: {info['title']}")
                    
                    formats = info['formats']
                    print("\nAvailable formats:")
                    for f in formats:
                        print(f"Format ID: {f['format_id']}, Resolution: {f.get('resolution', 'N/A')}, "
                              f"Extension: {f.get('ext', 'N/A')}, FPS: {f.get('fps', 'N/A')}, "
                              f"Video Codec: {f.get('vcodec', 'N/A')}, Audio Codec: {f.get('acodec', 'N/A')}")
                    
                    if quality != 'best':
                        if ffmpeg_available:
                            selected_format = next((f for f in formats if f['format_note'] == quality), None)
                            if selected_format:
                                ydl_opts['format'] = f"{selected_format['format_id']}+bestaudio/best"
                            else:
                                print(f"Requested quality '{quality}' not found. Defaulting to best available.")
                        else:
                            selected_format = next((f for f in formats if f['format_note'] == quality and f['acodec'] != 'none'), None)
                            if selected_format:
                                ydl_opts['format'] = selected_format['format_id']
                            else:
                                print(f"Requested quality '{quality}' not found. Defaulting to best available mp4 format.")
                                ydl_opts['format'] = 'best[ext=mp4]'
                    
                    print(f"\nSelected format: {ydl_opts['format']}")
                    ydl.params['format'] = ydl_opts['format']
                    ydl.download([url])
                
                print("Download completed successfully!")
            except Exception as e:
                print(f"An error occurred: {str(e)}")
    
    display(output)

def interactive_download():
    url_widget = widgets.Textarea(
        description='URL:',
        placeholder='Paste your YouTube URL here (video, playlist, or channel videos)',
        layout=Layout(width='50%', height='100px')
    )
    quality_widget = widgets.Text(description='Quality:', value='best')
    download_button = widgets.Button(description='Download')
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            output.clear_output()
            download_video(url_widget.value.strip(), quality_widget.value)

    download_button.on_click(on_button_clicked)
    display(widgets.VBox([url_widget, quality_widget, download_button, output]))

In [ ]:
login_youtube()


In [ ]:
interactive_download()